## 提示语模板

In [1]:
from illufly.types import Template, Messages, Runnable

t = Template("IDEA", binding={"task": "hi"})
t.consumer_dict

{'task': 'hi'}

In [2]:
t.format()

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\nhi\n'

### 通过 format 提供变量

In [3]:
t.format({"task": "帮我写一首儿歌"})

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n帮我写一首儿歌\n'

### 通过绑定 Runnable 提供变量

In [4]:
from illufly.types import Template, Messages, Runnable

class A(Runnable):
    def __init__(self):
        self.task = ""
    @property
    def provider_dict(self):
        return {"task": self.task}

    def call(self, task: str, **kwargs):
        self.task = task

a = A()
t = Template("IDEA")
t.bind_providers(a)

a.call("帮我写一首儿歌")
t.format()

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n帮我写一首儿歌\n'

### 绑定时使用映射

In [5]:
class A(Runnable):
    def call(self, prompt: str, **kwargs):
        self._last_input = prompt
        self._last_output = f"你说{prompt}？"

a = A()
t = Template("IDEA")
t.bind_providers((a, {"task": "last_input"}))

a.call("帮我写一首儿歌")
t.format()

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n帮我写一首儿歌\n'

## Messages 中使用提示语模板

In [6]:
from illufly.chat import FakeLLM
from illufly.types import Template

llm = FakeLLM(
    memory=[
        Template(text="请你帮我写{{mytask}}"),
        "请开始"
    ],
    template_binding={"mytask": "task"}
)

In [7]:
llm("一首诗词", verbose=True, new_chat=True)

[INFO] FakeLLM: [{'role': 'system', 'content': '请你帮我写一首诗词'}, {'role': 'user', 'content': '请开始'}]
这是一个模拟调用!

'这是一个模拟调用!'

In [8]:
llm.memory

[{'role': 'system', 'content': '请你帮我写一首诗词'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

In [11]:
llm.init_messages_bound_vars

{'last_output', 'mytask', 'task'}

In [10]:
for template in llm.init_messages.all_templates:
    print(template.consumer_dict)

{'last_input': '一首诗词', 'last_output': '这是一个模拟调用!', 'mytask': '一首诗词'}


In [11]:
llm.get_chat_memory()

[{'role': 'system', 'content': '请你帮我写一首诗词'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

In [14]:
from illufly.chat import ChatQwen
from illufly.types import Template

llm = ChatQwen(
    memory=[Template(text="请你帮我写{{mytask}}"), "请开始"],
    template_binding={
        "mytask": "task"
    }
)
llm("一首儿歌，4行")

小兔子乖乖，  
蹦蹦跳跳真快活，  
红萝卜甜甜，  
幸福日子每一天。


'小兔子乖乖，  \n蹦蹦跳跳真快活，  \n红萝卜甜甜，  \n幸福日子每一天。'

In [15]:
llm.memory

[{'role': 'system', 'content': '请你帮我写一首儿歌，4行'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '小兔子乖乖，  \n蹦蹦跳跳真快活，  \n红萝卜甜甜，  \n幸福日子每一天。'}]

In [16]:
llm("换成小狗")

小狗汪汪叫，
尾巴摇摇跟着跑，
骨头香又脆，
快乐时光不疲惫。


'小狗汪汪叫，\n尾巴摇摇跟着跑，\n骨头香又脆，\n快乐时光不疲惫。'

In [17]:
llm.memory

[{'role': 'system', 'content': '请你帮我写一首儿歌，4行'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '小兔子乖乖，  \n蹦蹦跳跳真快活，  \n红萝卜甜甜，  \n幸福日子每一天。'},
 {'role': 'user', 'content': '换成小狗'},
 {'role': 'assistant', 'content': '小狗汪汪叫，\n尾巴摇摇跟着跑，\n骨头香又脆，\n快乐时光不疲惫。'}]